# 3교시 1D CNN 시계열/센서 패턴 템플릿 (이상탐지/분류)
시험장에서 **파형/진동/센서 시계열 패턴**이 주어졌을 때, RNN 대신/또는 함께 사용할 수 있는 **1D CNN** 복붙 템플릿.

## ✅ 커버하는 케이스
- **A. 윈도우 단위 이진 분류(이상/정상)** (추천: PR-AUC/F1 + 임계값 정책)
- (옵션) 멀티클래스도 간단 확장 가능

## 🎯 언제 1D CNN을 쓰나?
- “짧은 구간의 국소 패턴(피크/진동 모양)”이 중요할 때
- RNN보다 **빠르고 안정적으로** 패턴을 잡고 싶을 때
- 윈도우 기반 라벨이 주어질 때(또는 만들 수 있을 때)

> 시험장에서 바꿀 것: `CSV_PATH`, `time_col`, `signal_cols`, `target_col`, `LOOKBACK`


## 0) Imports & 재현성

In [1]:
import os, random, warnings, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import average_precision_score, roc_auc_score, precision_recall_curve, f1_score, precision_score, recall_score
warnings.filterwarnings("ignore")

SEED=42
random.seed(SEED); np.random.seed(SEED)

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

## 1) 토이데이터 생성 (파형/센서 패턴 + 희귀 이상)
- 정상: sin + 약간의 잡음
- 이상: 특정 구간에 스파이크/주파수 변화/진폭 증가
- 라벨: 이상 윈도우(1), 정상(0)


In [2]:
def make_toy_wave(n_steps=60000, seed=42):
    rng=np.random.default_rng(seed)
    t=np.arange(n_steps)

    # 기본 파형(정상)
    base = np.sin(2*np.pi*t/50) + 0.4*np.sin(2*np.pi*t/17)
    noise = rng.normal(0, 0.25, n_steps)
    sig = base + noise

    # 외생 센서 2개(상관 신호)
    s2 = 0.5*np.sin(2*np.pi*t/60) + rng.normal(0,0.15,n_steps)
    s3 = rng.normal(0,0.35,n_steps)

    # 희귀 이상 이벤트: 구간 단위로 패턴 변화
    event = np.zeros(n_steps, dtype=int)
    n_events = 120
    for _ in range(n_events):
        start = rng.integers(200, n_steps-400)
        L = rng.integers(80, 200)
        event[start:start+L] = 1

        mode = rng.choice(["spike","amp","freq"])
        if mode=="spike":
            sig[start:start+L] += rng.normal(2.0, 0.5, L)
        elif mode=="amp":
            sig[start:start+L] *= rng.uniform(1.8, 2.4)
        else:  # freq change
            sig[start:start+L] = np.sin(2*np.pi*np.arange(L)/rng.integers(12,25)) + rng.normal(0,0.2,L)

        # 외생도 함께 흔들리게
        s2[start:start+L] += rng.normal(0.6, 0.2, L)
        s3[start:start+L] += rng.normal(0.8, 0.3, L)

    df=pd.DataFrame({
        "time": t,
        "signal_1": sig,
        "signal_2": s2,
        "signal_3": s3,
        "event": event
    })

    # 결측(아주 소량)
    m = rng.random(n_steps) < 0.002
    df.loc[m, "signal_1"] = np.nan
    return df

df = make_toy_wave()
df.head(), df["event"].mean()


(   time  signal_1  signal_2  signal_3  event
 0     0  0.076179  0.117401  0.395097      0
 1     1  0.009834  0.005572 -0.270844      0
 2     2  0.705781  0.333106 -0.109865      0
 3     3  0.961331  0.003118 -0.120792      0
 4     4  0.392289 -0.048759 -0.466981      0,
 np.float64(0.24353333333333332))

## 2) 설정/로딩
- 실제 시험 데이터면 CSV 로딩으로 교체
- `signal_cols`: 1개(단일 센서)든 여러 개든 가능
- 타깃: `event`(0/1) — 없으면 “라벨 생성” 또는 비지도 모델로 전환


In [3]:
CSV_PATH = None  # 예: "/content/sensor.csv"
time_col = "time"
signal_cols = ["signal_1","signal_2","signal_3"]
target_col = "event"   # 0/1

if CSV_PATH is not None:
    df = pd.read_csv(CSV_PATH)

print(df[signal_cols + [target_col]].head())
print("Pos ratio:", df[target_col].mean())


   signal_1  signal_2  signal_3  event
0  0.076179  0.117401  0.395097      0
1  0.009834  0.005572 -0.270844      0
2  0.705781  0.333106 -0.109865      0
3  0.961331  0.003118 -0.120792      0
4  0.392289 -0.048759 -0.466981      0
Pos ratio: 0.24353333333333332


## 3) 전처리: 결측 처리 + 스케일링(훈련 구간 fit)
1D CNN은 입력 스케일에 민감합니다. 스케일링은 거의 필수.


In [4]:
dfp = df.copy()
# 결측: 선형 보간 + ffill/bfill
dfp[signal_cols] = dfp[signal_cols].interpolate().ffill().bfill()

# 시간순 split
split = int(len(dfp)*0.8)
train_df = dfp.iloc[:split].copy()
val_df = dfp.iloc[split:].copy()

USE_ROBUST = True
scaler = RobustScaler() if USE_ROBUST else StandardScaler()

train_df[signal_cols] = scaler.fit_transform(train_df[signal_cols])
val_df[signal_cols] = scaler.transform(val_df[signal_cols])


## 4) 윈도우링(핵심)
- 입력 X: (N, C, T) 형태가 Conv1D에 가장 편함
  - C = 채널(센서 개수)
  - T = time length(LOOKBACK)
- 라벨 y:
  - 기본: **다음 시점의 event**
  - 또는 윈도우 내 event 존재 여부(window_any)로 라벨 완화(Recall 개선)


In [5]:
LOOKBACK = 128
LABEL_MODE = "window_any"  # "next" or "window_any"

def make_windows_1d(df_scaled, lookback=128):
    X_list=[]; y_list=[]; t_list=[]
    Xv = df_scaled[signal_cols].values
    yv = df_scaled[target_col].values.astype(int)
    tv = df_scaled[time_col].values

    for i in range(lookback, len(df_scaled)):
        seg = Xv[i-lookback:i]  # (lookback, C)
        # Conv1D expects (C, T)
        X_list.append(seg.T)
        if LABEL_MODE=="next":
            y_list.append(yv[i])
        else:
            y_list.append(int(yv[i-lookback:i].max()))
        t_list.append(tv[i])
    return np.stack(X_list), np.array(y_list), np.array(t_list)

Xtr, ytr, ttr = make_windows_1d(train_df, LOOKBACK)
Xva, yva, tva = make_windows_1d(val_df, LOOKBACK)

print("Xtr:", Xtr.shape, "ytr:", ytr.shape, "pos ratio:", ytr.mean())
print("Xva:", Xva.shape, "pos ratio:", yva.mean())


Xtr: (47872, 3, 128) ytr: (47872,) pos ratio: 0.41268382352941174
Xva: (11872, 3, 128) pos ratio: 0.4256233153638814


## 5) Dataset / DataLoader

In [6]:
class WindowDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1,1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

train_ds = WindowDataset(Xtr, ytr)
val_ds = WindowDataset(Xva, yva)

train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=512, shuffle=False)


## 6) 1D CNN 모델
구조(시험용 안정 버전):
- Conv1D → ReLU → Pool
- Conv1D → ReLU → Pool
- GlobalAvgPool(평균) 또는 Flatten
- FC → logit


In [7]:
C = Xtr.shape[1]  # channels

class CNN1D(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(C, 32, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),  # (B, 128, 1)
            nn.Flatten(),             # (B, 128)
            nn.Dropout(0.3),
            nn.Linear(128, 1)         # logit
        )
    def forward(self, x):
        z = self.conv(x)
        return self.head(z)

model = CNN1D(C).to(device)
model


CNN1D(
  (conv): Sequential(
    (0): Conv1d(3, 32, kernel_size=(7,), stride=(1,), padding=(3,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): ReLU()
  )
  (head): Sequential(
    (0): AdaptiveAvgPool1d(output_size=1)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=1, bias=True)
  )
)

## 7) Loss/Optimizer (불균형 대응)
- `pos_weight = neg/pos`로 양성(이상)에 가중치 부여
- 수치 안정: BCEWithLogitsLoss 사용


In [8]:
pos = int((ytr==1).sum()); neg = int((ytr==0).sum())
pos_weight = torch.tensor([neg / max(pos,1)], dtype=torch.float32, device=device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

print("pos_weight:", float(pos_weight.item()))


pos_weight: 1.423162579536438


## 8) 학습 루프 + PR-AUC 평가

In [9]:
def eval_val():
    model.eval()
    logits_all=[]; y_all=[]
    val_loss=0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb=xb.to(device); yb=yb.to(device)
            logits = model(xb)
            loss = criterion(logits, yb)
            val_loss += loss.item()*len(xb)
            logits_all.append(logits.cpu().numpy())
            y_all.append(yb.cpu().numpy())
    logits_all = np.vstack(logits_all).ravel()
    y_all = np.vstack(y_all).ravel()
    y_prob = 1/(1+np.exp(-logits_all))
    pr = average_precision_score(y_all, y_prob)
    roc = roc_auc_score(y_all, y_prob)
    return val_loss/len(val_ds), y_all, y_prob, pr, roc

best_pr=-1
best_state=None
EPOCHS=12

for epoch in range(1, EPOCHS+1):
    model.train()
    tr_loss=0.0
    for xb, yb in train_loader:
        xb=xb.to(device); yb=yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        tr_loss += loss.item()*len(xb)

    val_loss, y_true, y_prob, pr, roc = eval_val()
    if pr > best_pr:
        best_pr = pr
        best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}

    print(f"epoch {epoch:02d} | train_loss {tr_loss/len(train_ds):.4f} | val_loss {val_loss:.4f} | PR-AUC {pr:.4f} | ROC-AUC {roc:.4f}")

model.load_state_dict(best_state)
print("Best PR-AUC:", best_pr)


epoch 01 | train_loss 0.1457 | val_loss 0.0841 | PR-AUC 0.9934 | ROC-AUC 0.9924
epoch 02 | train_loss 0.0570 | val_loss 0.0655 | PR-AUC 0.9958 | ROC-AUC 0.9949
epoch 03 | train_loss 0.0363 | val_loss 0.0464 | PR-AUC 0.9978 | ROC-AUC 0.9974
epoch 04 | train_loss 0.0236 | val_loss 0.0409 | PR-AUC 0.9988 | ROC-AUC 0.9987
epoch 05 | train_loss 0.0174 | val_loss 0.0273 | PR-AUC 0.9992 | ROC-AUC 0.9993
epoch 06 | train_loss 0.0133 | val_loss 0.0184 | PR-AUC 0.9996 | ROC-AUC 0.9996
epoch 07 | train_loss 0.0122 | val_loss 0.0193 | PR-AUC 0.9996 | ROC-AUC 0.9996
epoch 08 | train_loss 0.0097 | val_loss 0.0157 | PR-AUC 0.9997 | ROC-AUC 0.9997
epoch 09 | train_loss 0.0095 | val_loss 0.0157 | PR-AUC 0.9997 | ROC-AUC 0.9997
epoch 10 | train_loss 0.0082 | val_loss 0.0131 | PR-AUC 0.9998 | ROC-AUC 0.9997
epoch 11 | train_loss 0.0078 | val_loss 0.0139 | PR-AUC 0.9998 | ROC-AUC 0.9998
epoch 12 | train_loss 0.0073 | val_loss 0.0149 | PR-AUC 0.9998 | ROC-AUC 0.9998
Best PR-AUC: 0.9997859322213243


## 9) 임계값 정책 + 리포트(제출용)
- F1-opt
- Top 5%
- Recall 고정(예: 0.90)


In [10]:
# 최종 확률
model.eval()
with torch.no_grad():
    logits = model(torch.tensor(Xva, dtype=torch.float32, device=device)).cpu().numpy().ravel()
y_prob = 1/(1+np.exp(-logits))
y_true = yva.astype(int)

print("PR-AUC:", average_precision_score(y_true, y_prob))
print("ROC-AUC:", roc_auc_score(y_true, y_prob))

prec, rec, ths = precision_recall_curve(y_true, y_prob)
f1s = 2*prec*rec/(prec+rec+1e-12)

idx = np.argmax(f1s)
th_f1 = ths[idx] if idx < len(ths) else 0.5
th_top5 = np.percentile(y_prob, 95)

TARGET_RECALL = 0.90
valid = np.where(rec[:-1] >= TARGET_RECALL)[0]
th_recall = th_f1
if len(valid) > 0:
    idx_c = valid[np.argmax(prec[valid])]
    th_recall = ths[idx_c]

def eval_at(th):
    y_pred = (y_prob >= th).astype(int)
    return {
        "threshold": float(th),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "alarm_rate": float(y_pred.mean())
    }

print("F1-opt:", eval_at(th_f1))
print("Top5% :", eval_at(th_top5))
print("Recall>=%.2f:"%TARGET_RECALL, eval_at(th_recall))

report = pd.DataFrame({
    "time": tva,
    "score": y_prob,
    "y_true": y_true
}).sort_values("score", ascending=False)

display(report.head(30))


PR-AUC: 0.9997859322213243
ROC-AUC: 0.9997773128231275
F1-opt: {'threshold': 0.26715087890625, 'precision': 0.9996024647187438, 'recall': 0.9952503463289135, 'f1': 0.9974216580721936, 'alarm_rate': 0.42377021563342315}
Top5% : {'threshold': 1.0, 'precision': 1.0, 'recall': 0.9576489214328122, 'f1': 0.9783663566518399, 'alarm_rate': 0.40759770889487873}
Recall>=0.90: {'threshold': 0.6844416856765747, 'precision': 1.0, 'recall': 0.9930734217296655, 'f1': 0.9965246748088571, 'alarm_rate': 0.42267520215633425}


,time,score,y_true
11223,59351,1.0,1
11222,59350,1.0,1
11221,59349,1.0,1
11220,59348,1.0,1
11219,59347,1.0,1
11218,59346,1.0,1
11217,59345,1.0,1
11216,59344,1.0,1
31,48159,1.0,1
30,48158,1.0,1


## 10) 실무 엔지니어 플레이북(1D CNN)
- **Recall 낮음(놓침)**: `LABEL_MODE="window_any"` / threshold↓ / pos_weight↑(또는 더 큰 값으로 수동 설정)
- **Precision 낮음(오탐)**: threshold↑ / Top-N 정책 / 규제(dropout↑, weight_decay↑)
- **과적합**: Conv 채널 수 줄이기, dropout↑, epoch↓
- **데이터가 멀티센서**: 채널(C)로 그대로 넣는 게 자연스럽고, 센서별 스케일링이 특히 중요
